In [1]:
import pandas as pd
import torch
import numpy as np

D:\Users\Boris\AppData\Local\Temp\ipykernel_18388\2758619612.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def check_in_dictionary(word, dictionary):
    return word in dictionary

# Функция для исправления опечаток в слове
def correct_spelling(word, dictionary):
    possible_corrections = []
    for dict_word in dictionary:
        if len(dict_word) == len(word):
            diff_count = sum(1 for a, b in zip(dict_word, word) if a != b)
            if diff_count <= 1:
                possible_corrections.append(dict_word)
    return possible_corrections

# Преобразование слова в вектор
def word_to_tensor(word, vocab):
    tensor = torch.zeros(len(word), dtype=torch.long)
    for i, char in enumerate(word):
        tensor[i] = vocab[char]
    return tensor


In [4]:
with open('dict.txt', 'r', encoding='utf-8') as f:
    dictionary = set(word.strip() for word in f.readlines())

# Загрузка списка запросов с ошибками
with open('queries.txt', 'r', encoding='utf-8') as f:
    queries = [word.strip() for word in f.readlines()]


In [5]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
vocab = {char: idx for idx, char in enumerate(alphabet)}

results = []
for query in queries:
    corrections = correct_spelling(query, dictionary)
    if check_in_dictionary(query, dictionary):
        results.append(f'"{query}" 0')
    elif corrections:
        result_str = f'"{query}"'
        for correction in corrections:
            result_str += f' 1 "{correction}"'
        results.append(result_str)
    else:
        results.append(f'"{query}" 3+')

In [6]:
df = pd.DataFrame(results, columns=['Результат проверки'])
df.to_csv('spell_check_results.txt', index=False, header=False)

In [7]:
df

,Результат проверки
0,"""воспитаннясть"" 1 ""воспитанность"""
1,"""неоплатоэивк"" 3+"
2,"""махаьный"" 3+"
3,"""втникж"" 3+"
4,"""эйдетки"" 3+"
...,...
99995,"""ккибкосинг"" 3+"
99996,"""переусердствование"" 0"
99997,"""жерло"" 0"
99998,"""отдерыгивание"" 3+"


In [8]:
with open('spell_check_results.txt', 'r', encoding='utf-8') as f:
    data = f.read()
    
    data_clear = data.replace('"', '')
    with open('out.txt', 'w', encoding='utf-8') as f:
        f.write(data_clear)